<a href="https://colab.research.google.com/github/Nagawade/Fake-News-Classification/blob/main/Fake_News_detetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('/content/drive/MyDrive/WELFake_Dataset.csv.xlsx')

In [3]:
df.head(5)

,Unnamed: 0,title,text,label,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,Did they post their votes for Hillary already?,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,UNBELIEVABLE! OBAMAâ€™S ATTORNEY GENERAL SAYS ...,"Now, most of the demonstrators gathered last ...",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(78097, 686)

In [5]:

unnamed_cols = [col for col in df.columns if 'Unnamed' in col]

df = df.drop(columns=unnamed_cols)

df.shape

(78097, 3)

In [6]:
df.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMAâ€™S ATTORNEY GENERAL SAYS ...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [7]:
df.isnull().sum()

,0
title,2961
text,3609
label,4321


In [8]:
df = df.dropna()

In [9]:
df.shape

(73177, 3)

In [10]:
X = df.drop(columns='label')
y = df['label']
# Convert 'label' column to numeric, coercing errors
y = pd.to_numeric(y, errors='coerce')

In [11]:
X.shape

(73177, 2)

In [12]:
y.shape

(73177,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.18.0'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
voc_size = 5000

# One Hot Encoding

In [17]:
message = X.copy()
message.reset_index(drop=True, inplace=True)

In [18]:
message

,title,text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
1,UNBELIEVABLE! OBAMAâ€™S ATTORNEY GENERAL SAYS ...,"Now, most of the demonstrators gathered last ..."
2,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
...,...,...
73172,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...
73173,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n..."
73174,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...
73175,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trumpâ€™s comba...


# Preprocessing of data

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [22]:
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ',str(message['title'][i]))
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [23]:
corpus

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video',
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group',
 'dr ben carson target ir never audit spoke nation prayer breakfast',
 'hous intel chair trump russia fake stori evid anyth video',
 'sport bar owner ban nfl game show true american sport like speak rural america video',
 'latest pipelin leak underscor danger dakota access pipelin',
 'gop senat smack punchabl alt right nazi internet',
 'may brexit offer would hurt cost eu citizen eu parliament',
 'schumer call trump appoint offici overse puerto rico relief',
 'watch hilari ad call question health age clinton crime famili boss',
 'chang expect espn polit agenda despit huge subscrib declin breitbart'

In [24]:
#One hot encoding

onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[3840, 328, 1738, 2900, 2289, 2736, 4307, 4023, 508, 3633, 1804, 1914],
 [3038,
  491,
  2069,
  3753,
  3458,
  423,
  2776,
  4792,
  3879,
  1298,
  2180,
  2675,
  2034,
  1914],
 [2734, 1613, 3326, 399, 3648, 3975, 764, 1552, 4179, 2662, 4506, 4022],
 [2210, 1122, 4192, 4627, 1831, 1161, 2950, 109, 3621, 3461, 4296],
 [3665, 764, 3289, 1827, 1781, 3018, 93, 2523, 3289],
 [3020, 506, 3761, 2110, 4774, 3228, 2903, 1621, 2804, 332, 2255],
 [3145, 1800, 3390, 1357, 1122, 440, 3975, 1897, 4214, 1914],
 [2338,
  3886,
  1485,
  325,
  1589,
  1718,
  4574,
  4705,
  4638,
  2338,
  3333,
  674,
  2872,
  2548,
  1914],
 [3116, 382, 2436, 2429, 1150, 3177, 1258, 382],
 [2832, 1510, 590, 4433, 1269, 1869, 4665, 3602],
 [1088, 2618, 1476, 344, 4673, 4523, 1407, 3351, 1407, 3376],
 [2710, 678, 1357, 4640, 3055, 3470, 895, 37, 1117],
 [1733, 3817, 1560, 678, 3158, 1262, 610, 4012, 1683, 744, 548],
 [3750, 4606, 4368, 4433, 537, 3588, 4771, 3330, 567, 226],
 [3730, 507, 267, 4948, 4419, 3145

In [25]:
onehot_repr[0]

[3840, 328, 1738, 2900, 2289, 2736, 4307, 4023, 508, 3633, 1804, 1914]

Word to vec representetion

In [26]:
sent_len = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_len)


In [27]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0, 3038,  491, 2069, 3753, 3458,
        423, 2776, 4792, 3879, 1298, 2180, 2675, 2034, 1914], dtype=int32)

# Model Building

In [29]:
model = Sequential()
embeding_vector_features = 40 # Defining the variable
model.add(Embedding(input_dim=voc_size, output_dim=embeding_vector_features, input_shape=(sent_len,)))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
len(embedded_docs), y.shape

(73177, (73177,))

In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set your desired max length
sent_len = 20

x_final = pad_sequences(embedded_docs, maxlen=sent_len, padding='post')




In [34]:
print(x_final.shape)         # Should be (samples, sent_len)
print(x_final.dtype)         # Should be int32 or int64


(73177, 20)
int32


In [35]:
# y_train should now be numeric after dropping NaNs and splitting
# No need to explicitly convert y_train here

In [36]:
from sklearn.model_selection import train_test_split
# Drop rows where y is NaN before splitting
X_train, X_test, y_train, y_test = train_test_split(x_final[~y.isna()], y[~y.isna()], test_size=0.2, random_state=42)

In [37]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)


Epoch 1/10
893/893 ━━━━━━━━━━━━━━━━━━━━ 38s 40ms/step - accuracy: 0.7225 - loss: -4.3806 - val_accuracy: 0.6360 - val_loss: 1.7034
Epoch 2/10
893/893 ━━━━━━━━━━━━━━━━━━━━ 33s 37ms/step - accuracy: 0.7096 - loss: -1.7334 - val_accuracy: 0.6864 - val_loss: 1.5544
Epoch 3/10
893/893 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.7804 - loss: -3.2459 - val_accuracy: 0.8068 - val_loss: -0.2722
Epoch 4/10
893/893 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.8164 - loss: -134.2057 - val_accuracy: 0.8270 - val_loss: -0.5390
Epoch 5/10
893/893 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.8465 - loss: -147.7599 - val_accuracy: 0.8623 - val_loss: -0.7222
Epoch 6/10
893/893 ━━━━━━━━━━━━━━━━━━━━ 32s 36ms/step - accuracy: 0.8829 - loss: -166.4483 - val_accuracy: 0.8657 - val_loss: -0.8518
Epoch 7/10
893/893 ━━━━━━━━━━━━━━━━━━━━ 41s 37ms/step - accuracy: 0.9019 - loss: -11.8159 - val_accuracy: 0.8904 - val_loss: -0.9678
Epoch 8/10
893/893 ━━━━━━━━━━━━━━━━━━━━ 41s 37ms/step - accuracy: 0.915

# Model Testing

In [38]:
y_pred = model.predict(X_test)

447/447 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step


In [39]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8968292853643172

In [41]:
from sklearn.metrics import confusion_matrix

In [42]:
cf = confusion_matrix(y_test, y_pred)
cf

array([[6207,  752,    0,    0],
       [ 720, 6606,    0,    0],
       [   0,    1,    0,    0],
       [   0,    1,    0,    0]])

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.89      0.89      6959
         1.0       0.90      0.90      0.90      7326
        37.0       0.00      0.00      0.00         1
       500.0       0.00      0.00      0.00         1

    accuracy                           0.90     14287
   macro avg       0.45      0.45      0.45     14287
weighted avg       0.90      0.90      0.90     14287



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
